In [1]:
# Run to load all the Libraries and helper functions
%run "Utilities/libraries.py"

In [2]:
# INPUTS TO THE SCRIPTS
Bucket_Name="model-deploy-poc-new"

train_test_val_location_S3="train_test_val"


text_column="Article"
target_column="Summary"

In [3]:
model_names="patrickvonplaten/bert2bert_cnn_daily_mail"

In [4]:
sess=boto3.session.Session()
role=sagemaker.get_execution_role()

In [5]:
input_path = f"s3://{Bucket_Name}/{train_test_val_location_S3}"


## 4. Invoke Model Training

In [9]:
import sagemaker
from sagemaker.huggingface import HuggingFace

# Experiment with your models here. Change hyperparameters to optimize your results
def model_invoke(model_name,train_test_val_location_S3):
    output_dir_name=model_name.split("/")[-1]
    output_path = f"s3://{Bucket_Name}/model/{output_dir_name}"
    # gets role for executing training job
    role = sagemaker.get_execution_role()
    hyperparameters = {
        "model-name": model_name,
        "text-column": text_column,
        "target-column": target_column,
        "epoch": 5,
        'train-data-dir':input_path,
        'log-dir':output_path+"/Logs",
        'train-batch-size': 5,
        'eval-batch-size': 2,
     # more info here https://github.com/huggingface/transformers/tree/v4.17.0/examples/pytorch/summarization
    }

    metric_definitions = [
        {"Name": "training:loss", "Regex": "'loss': (.*?),"},
        {"Name": "validation:loss", "Regex": "'eval_loss': (.*?),"},
        {"Name": "validation:rouge1", "Regex": "'eval_rouge1': (.*?),"},
        {"Name": "validation:rouge2", "Regex": "'eval_rouge2': (.*?),"},
        {"Name": "validation:rougeL", "Regex": "'eval_rougeL': (.*?),"},
        {"Name": "validation:rougeLsum", "Regex": "'eval_rougeLsum': (.*?),"},
        {"Name": "validation:gen_len", "Regex": "'eval_gen_len': (.*?),"},
    ]

    # git configuration to download our fine-tuning script
    # git_config = {'repo': 'https://github.com/huggingface/transformers.git','branch': 'v4.17.0'}

# creates Hugging Face estimator
    huggingface_estimator = HuggingFace(
     entry_point='train.py',
     source_dir='Utilities',
     instance_type='ml.p3.2xlarge',
     base_job_name=(model_name.split("/")[-1]).replace("_","-"),   
     instance_count=1,
     role=role,
    #  git_config=git_config,
     transformers_version='4.17.0',
     pytorch_version='1.10.2',
     py_version='py38',
     hyperparameters = hyperparameters,
     output_path=output_path,
#  If operating on ml.p3.4xlarge and above we can opt for distributed computing to reduce training time.
#      distribution ={"mpi": { "enabled": True },"smdistributed": {"modelparallel": { "enabled": True,"parameters": {}}}},
     metric_definitions=metric_definitions,
    )
    
    huggingface_estimator.fit({"train": f"s3://{Bucket_Name}/{train_test_val_location_S3}/train",
                           "test": f"s3://{Bucket_Name}/{train_test_val_location_S3}/val"})
    return(huggingface_estimator)
# starting the train job
# huggingface_estimator.fit()

In [ ]:
model=model_invoke(model_name=model_names,train_test_val_location_S3=train_test_val_location_S3)

2022-07-03 11:41:42 Starting - Starting the training job...
2022-07-03 11:42:05 Starting - Preparing the instances for trainingProfilerReport-1656848501: InProgress
.........
2022-07-03 11:43:25 Downloading - Downloading input data...
2022-07-03 11:44:05 Training - Downloading the training image.......................
2022-07-03 11:48:46 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
/opt/conda/lib/python3.8/site-packages/paramiko/transport.py:236: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,
2022-07-03 11:48:35,969 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-07-03 11:48:35,990 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-07-03 11:48:35,998 sagemaker_pytorch_container.training INFO     Invoking user training s

In [ ]:
# {j:model_dict[j].model_data for i,j in enumerate(model_dict)}

In [ ]:
from sagemaker.huggingface.model import HuggingFaceModel

model_name = "bert2bert_cnn_daily_mail"
# Enter final location of your Model
model_location=f"s3://{Bucket_Name}/final_models/"+model_name+".tar.gz"

model_for_deployment = HuggingFaceModel(
    entry_point="inference.py",
    source_dir="Utilities",
    model_data=model_location,
    role=role,
    pytorch_version="1.7.1",
    py_version="py36",
    transformers_version="4.6.1",
    name=model_name.replace(r"_","-")+"-V2",
)

In [ ]:
endpoint_name = "summarization-endpoint-5"+model_name+"-1"

predictor = model_for_deployment.deploy(
    initial_instance_count=1,
    instance_type="ml.g4dn.xlarge",
    endpoint_name=endpoint_name.replace(r"_","-"),
    serializer=sagemaker.serializers.JSONSerializer(),
    deserializer=sagemaker.deserializers.JSONDeserializer(),
)

In [ ]:
predictor.predict({
'inputs': "The tower is 324 metres (1,063 ft) tall, about the same height as an 81-storey building, and the tallest structure in Paris. Its base is square, measuring 125 metres (410 ft) on each side. During its construction, the Eiffel Tower surpassed the Washington Monument to become the tallest man-made structure in the world, a title it held for 41 years until the Chrysler Building in New York City was finished in 1930. It was the first structure to reach a height of 300 metres. Due to the addition of a broadcasting aerial at the top of the tower in 1957, it is now taller than the Chrysler Building by 5.2 metres (17 ft). Excluding transmitters, the Eiffel Tower is the second tallest free-standing structure in France after the Millau Viaduct."
})

### Average first loading time 2 Seconds

In [ ]:
# Can opt for Async inferencing for higher load
# from sagemaker.async_inference import AsyncInferenceConfig
# prefix="async-location"
# endpoint_name = "summarization-endpoint-5"+model_name+"-async-V1"
# # Create an empty AsyncInferenceConfig object to use default values
# async_config = AsyncInferenceConfig(output_path=f"s3://{Bucket_Name}/{prefix}/output")

# # deploy model to SageMaker Inference
# async_predictor = model_for_deployment.deploy(
#     async_inference_config=async_config,
#     initial_instance_count=1, # number of instances
#     instance_type='ml.g4dn.xlarge', # instance type,
#     serializer=sagemaker.serializers.JSONSerializer(),
#     deserializer=sagemaker.deserializers.JSONDeserializer(),
#     endpoint_name=endpoint_name.replace(r"_","-"),
# )

In [ ]:
# async_predictor.predict_async({
# 'inputs': "The tower is 324 metres (1,063 ft) tall, about the same height as an 81-storey building, and the tallest structure in Paris. Its base is square, measuring 125 metres (410 ft) on each side. During its construction, the Eiffel Tower surpassed the Washington Monument to become the tallest man-made structure in the world, a title it held for 41 years until the Chrysler Building in New York City was finished in 1930. It was the first structure to reach a height of 300 metres. Due to the addition of a broadcasting aerial at the top of the tower in 1957, it is now taller than the Chrysler Building by 5.2 metres (17 ft). Excluding transmitters, the Eiffel Tower is the second tallest free-standing structure in France after the Millau Viaduct."
# })

In [ ]:
# predictor.delete_endpoint()